# Analysis

In [ ]:
import os
import glob
import json
import pickle
from collections import defaultdict

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']

In [ ]:
results = []
for filename in glob.glob("results/*.pkl"):
    with open(filename, "rb") as f:
        results.append(pickle.load(f))
print(len(results))
print(results[0].keys())
print(results[0]["times"].keys())
print(results[0]["evaluate"].keys())
print(results[0]["evaluate"]["R0"].keys())
print(results[0]["infer"].keys())
print(results[0]["infer"]["R0"].keys())

In [ ]:
def plot_accuracy(variable, metric):
    view = defaultdict(list)
    for result in results:
        args = result['args']
        view[args.infer, args.num_bins, args.svi_steps].append(result)
    markers = ["o", "d", "s", "<", "v", "^", ">"]
    assert len(view) == len(markers)

    plt.figure(figsize=(6, 5)).patch.set_color("white")
    for (key, value), marker in zip(sorted(view.items()), markers):
        algo, num_bins, svi_steps = key
        if algo == "svi":
            label = "SVI steps={}".format(svi_steps)
        elif algo == "mcmc":
            if num_bins == 1:
                label = "MCMC relaxed"
            else:
                label = "MCMC num_bins={}".format(num_bins)
        X = [v["times"]["infer"] for v in value]
        Y = [v["evaluate"][variable][metric] for v in value]
        plt.scatter(X, Y, marker=marker, label=label, alpha=0.8)
    plt.ylim(0, None)
    plt.xscale("log")
    plt.xlabel("inference time (sec)")
    plt.ylabel(metric)
    plt.title(f"{variable} accuracy")
    plt.legend(loc="best", prop={'size': 8})
    plt.tight_layout()

In [ ]:
plot_accuracy("R0", "crps")

In [ ]:
plot_accuracy("rho", "crps")

In [ ]:
plot_accuracy("obs", "crps")

In [ ]:
plot_accuracy("I", "crps")

In [ ]:
def plot_convergence(variable, metrics=["n_eff", "r_hat"]):
    view = defaultdict(list)
    for result in results:
        args = result['args']
        if args.infer == "mcmc":
            view[args.num_bins].append(result)
    markers = ["o", "d", "s"]
    assert len(view) == len(markers)

    fig, axes = plt.subplots(len(metrics), 1, figsize=(6, 5), sharex=True)
    fig.patch.set_color("white")
    for (num_bins, value), marker in zip(sorted(view.items()), markers):
        if num_bins == 1:
            label = "MCMC relaxed"
        else:
            label = "MCMC num_bins={}".format(num_bins)
        X = [v["times"]["infer"] for v in value]
        for metric, ax in zip(metrics, axes):
            Y = [v["infer"][variable][metric] for v in value]
            ax.scatter(X, Y, marker=marker, label=label, alpha=0.8)
            ax.set_xscale("log")
            ax.set_ylabel(metric)
    axes[0].set_title(f"{variable} convergence")
    axes[-1].legend(loc="best", prop={'size': 8})
    axes[-1].set_xlabel("inference time (sec)")
    plt.subplots_adjust(hspace=0)

In [ ]:
plot_convergence("R0")

In [ ]:
plot_convergence("rho")

In [ ]:
plot_convergence("auxiliary_haar_split_0")